In [30]:
import torch
from src.resnet import ResNet

In [31]:
model = ResNet.from_state_dict(torch.load('runs/baseline_gr-baseline_resnet_2024-01-18_21-49/checkpoint/resnet_ep1_acc100_sprs0_.pt'))

In [32]:
inp = torch.rand(1, *model.in_shape)
print(model(inp))
model.model_size()

tensor([[ 1.9393, -1.4409, -1.2533, -1.2056, -1.0202, -1.5568, -1.1396, -1.0405,
         -1.1839, -0.8565]], grad_fn=<AddmmBackward0>)


44.990297

In [33]:
model_int8 = model.quantize(qat=True, inplace=True)
print(model_int8(inp))
model_int8.model_size()

tensor([[ 0.1923, -0.0970, -0.1289, -0.0803, -0.1056, -0.0697, -0.0812,  0.0094,
         -0.0755, -0.0614]], grad_fn=<AddmmBackward0>)


44.990297